In [37]:
%%latex
string


<IPython.core.display.Latex object>

In [1]:
import numpy as np

In [27]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [19]:
#Define array

def symmetric_array_n(n):
    
    A = np.zeros((n, n))

    for i in range(0, n):
        for j in range(0, n):
            if i == j:
                A[i, j] = 2
            if np.abs(i-j) == 1:
                A[i, j] = -1

    return A

In [39]:
A = symmetric_array_n(10)
b = np.matrix([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).transpose()
X = np.linalg.lstsq(A, b)[0]

print bmatrix(A)

\begin{bmatrix}
  2. & -1. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0.\\
  -1. & 2. & -1. & 0. & 0. & 0. & 0. & 0. & 0. & 0.\\
  0. & -1. & 2. & -1. & 0. & 0. & 0. & 0. & 0. & 0.\\
  0. & 0. & -1. & 2. & -1. & 0. & 0. & 0. & 0. & 0.\\
  0. & 0. & 0. & -1. & 2. & -1. & 0. & 0. & 0. & 0.\\
  0. & 0. & 0. & 0. & -1. & 2. & -1. & 0. & 0. & 0.\\
  0. & 0. & 0. & 0. & 0. & -1. & 2. & -1. & 0. & 0.\\
  0. & 0. & 0. & 0. & 0. & 0. & -1. & 2. & -1. & 0.\\
  0. & 0. & 0. & 0. & 0. & 0. & 0. & -1. & 2. & -1.\\
  0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & -1. & 2.\\
\end{bmatrix}


In [42]:
%%latex

\begin{bmatrix}
  2. & -1. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0.\\
  -1. & 2. & -1. & 0. & 0. & 0. & 0. & 0. & 0. & 0.\\
  0. & -1. & 2. & -1. & 0. & 0. & 0. & 0. & 0. & 0.\\
  0. & 0. & -1. & 2. & -1. & 0. & 0. & 0. & 0. & 0.\\
  0. & 0. & 0. & -1. & 2. & -1. & 0. & 0. & 0. & 0.\\
  0. & 0. & 0. & 0. & -1. & 2. & -1. & 0. & 0. & 0.\\
  0. & 0. & 0. & 0. & 0. & -1. & 2. & -1. & 0. & 0.\\
  0. & 0. & 0. & 0. & 0. & 0. & -1. & 2. & -1. & 0.\\
  0. & 0. & 0. & 0. & 0. & 0. & 0. & -1. & 2. & -1.\\
  0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & -1. & 2.\\
\end{bmatrix}


<IPython.core.display.Latex object>